In [1]:
# Import dependencies
import pandas as pd
import os
import glob
import numpy as np
import datetime 

In [3]:
# Use glob to get all the supply data csv files in the current working directory
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))

In [4]:
# Create empty list
dataframes_list = []

In [5]:
# Append datasets to the list 
for i in csv_files:
    temp_df = pd.read_csv(i)
    dataframes_list.append(temp_df)

In [6]:
# Check dataframes_list length for all suppy data CSV files
len(dataframes_list)

1362

In [7]:
# Create empty DataFrame for all supply data
supply_df = pd.DataFrame()

In [63]:
# Transform each DataFrame in list
for df in dataframes_list:
    # Create date variable by splitting index position 0 column name
    date = df.columns[0].split("Supply", 0).pop()
    
    # Reset index to energy sources and transpose
    df_transposed = df.set_index(date).T
    
    # Create date column
    df_transposed["Date"] = date
    
    # Create time column 
    df_transposed["Time"] = df_transposed.index

    # Reset index to start from 0 and remove index name
    df_transposed.reset_index(drop=True, inplace=True)
    df_transposed.rename_axis(None, axis=1, inplace=True)
    
    # Cast 'Time' column to string and append seconds
    df_transposed['Time'] = df_transposed['Time'].apply(str) + ':00'
    df_transposed.iloc[0:120, 9] = "0" + df_transposed.iloc[0:120, 9]
    
    # Add 'Date' and 'Time' columns together
    df_transposed['DateTime'] = df_transposed['Date'] + 'T' + df_transposed['Time']
    
    try:
        # Covert 'DateTime' column to datetime 
        df_transposed['DateTime'] =  pd.to_datetime(df_transposed['DateTime'])
    except:
        print(df)
    
    # Create 'Unix Timestamp' column
    df_transposed['Unix Timestamp'] = df_transposed.DateTime.apply(lambda x : (x-datetime.datetime(1970,1,1)).total_seconds())
    
    # Convert 'Unix Timestamp' column to int
    #df_transposed['Unix Timestamp'].astype(np.int64)
    
    # Append to master DataFrame with all supply data
    #supply_df = supply_df.append(df_transposed, ignore_index = True) 

  Supply 12/02/2018  0:00  0:05  0:10  0:15  0:20  0:25  0:30  0:35  0:40  \
0        Renewables  3874  3897  3912  3883  3863  3861  3891  3888  3883   
1       Natural gas  9549  9721  9819  9850  9795  9714  9753  9708  9637   
2       Large hydro   943   990   988   968  1023   969   900   920   926   
3           Imports  7001  6736  6651  6582  6449  6493  6590  6568  6457   
4         Batteries    29    34    52    55    67    11   -28   -31     1   
5           Nuclear   505   505   506   506   506   507   506   507   507   
6              Coal    16    17    17    17    18    17    17    17    16   
7             Other     0     0     0     0     0     0     0     0     0   

   ...  23:10  23:15  23:20  23:25  23:30  23:35  23:40  23:45  23:50  23:55  
0  ...   2389   2376   2353   2318   2282   2256   2232   2208   2194   2174  
1  ...  10360  10356  10271  10214  10213  10131  10097  10064   9981  10008  
2  ...   1293   1255   1223   1182   1184   1140   1134   1149   1134

TypeError: unsupported operand type(s) for -: 'str' and 'datetime.datetime'